# Seeme Quick Guide - Fastai v2

This is a quick guide to deploy your trained models using [SeeMe.ai](https://seeme.ai).

In this guide, we will create, deploy and use a simple Cats and Dogs classifier. 

The focus is getting you up and running with deploying, using and sharing your AI models. For the ins and outs of training AI models with Fast.ai, we recommend you follow the excellent [Fast.ai courses](fast.ai).

# 0. Disclaimer

* Fastai v2 is currently in development and things might break.
* We will try our best to support it throughout its development to a stable version.
* SeeMe.ai is currently in development and things might break:
    * For the fastai v2 beta period, you can host one model with up to 500 predictions for free;
    * We are working on a fair and sustainable pricing (contact [iwantto@seeme.ai](mailto:iwantto@seeme.ai) for more info);
    * We rely on your code of honor for fair use.

# 0. Setup

1. Use your preferred platfrom to run your [Jupyter Notebooks](https://jupyter.org/): 

    * [Paperspace](https://paperspace.com)
    * [Google Colab](https://colab.research.google.com/)
    * [Docker](https://hub.docker.com/r/zerotosingularity/fastai2)
    * [Azure](https://azure.microsoft.com)
    * ...

In [ ]:
# See if you have fastai v2 installed:
import fastai2

# this version should correspond to the one on: https://pypi.org/project/fastai2/
print(fastai2.__version__)

2. Install the [SeeMe SDK](https://pypi.org/project/seeme/) from the command line:

```bash
$ pip install seeme
```

or in your Notebook:

In [ ]:
# Install the SeeMe SDK from your notebook.
!pip install seeme

# 1. Train a model

Here we will train a basic Cats vs Dogs model, using the defaults from fastai.

In [ ]:
# Import the vision modules
from fastai2.vision.all import *

In [ ]:
# get the `catsdogs` dataset
path = untar_data(URLs.DOGS)

In [ ]:
# have a look at the folder
path.ls()

In [ ]:
# Create the dataloaders
# for more details check the fast.ai courses
dls = ImageDataLoaders.from_folder(path, seed=12, item_tfms=Resize(460),
                                    batch_tfms=aug_transforms(size=224))

In [ ]:
# Create your model
learn =cnn_learner(dls, resnet50, metrics=error_rate)

In [ ]:
# Fine tune your model
learn.fine_tune(5)

In [ ]:
image_location = path/"test1/10789.jpg"

In [ ]:
# Find an image to test our model with
img = PILImage.create(image_location)

In [ ]:
# Pretty clear for us humans
img

In [ ]:
# Let's see what our model thinks
learn.predict(img)

# 2. Deployment - SeeMe.ai

Once you have trained your model, it's time to get it into production.

In [ ]:
# Export your model
learn.export()

In [ ]:
# Import the SeeMe client
from seeme import Client

In [ ]:
# Create a SeeMe client
client = Client()

In [ ]:
# If you haven't already registered, follow these steps:
my_password =  # example: "supersecurepasswordfromonepassword"
my_username =  # example: "janvdp"
my_email =   # example: "jan@seeme.ai"
my_firstname =   # example: "Jan"
my_name =  # example: "Van de Poel"

client.register(username=my_username, email=my_email, password=my_password, firstname=my_firstname, name=my_name)

In [ ]:
# Once you have registered, you can log in:
client.login(my_username, my_password)

In [ ]:
# Create model with name and description

model_name = "My first model"
description = "Created to be used..."
classes = "Cats and dogs"

my_model = client.create_full_model({
    "name": model_name,
    "description": description,
    "classes": classes,
    "framework": "fastai",
    "version": "2"
})

In [ ]:
# Deploy your exported model.
client.upload_model(my_model["ID"], str(path))

# By default this takes the export.pkl created by learn.export(), but you can simply pass in a filename as well.

In [ ]:
# Upload a logo for your model (optional)
client.upload_logo(my_model["ID"], folder=str(path/"test1"), filename="1622.jpg")

# 3. Use your model

Once your model is deployed, you can use it in a number of ways:

- Python SDK (see below)
- [SeeMe web app](https://app.seeme.ai)
- iOS app (Testflight invite)
- Android (wip)

## 3.1 Python SDK

Just like you can make predictions on the model itself, you can now use the SeeMe Python SDK to make predictions from basically anywhere, provided you have:

- SeeMe SDK installed
- Login credentials
- The ID of a deployed model
- An image to classiy

😃


In [ ]:
# Here, we will use the Python SDK to classify our test image
result = client.inference(my_model["ID"], image_location)

In [ ]:
# Print out the results
print(result["prediction"])
print(result["confidence"])

Make sure to check if they match the predictions from our model in the notebook.

## 3.2 SeeMe.ai web app

You can open the web app via [app.seeme.ai](https://app.seeme.ai)

Log in if you need to with the credentials used to register (my_username, my_password).

If you've followed the steps above without changing anything, this is what you will see after logging in.

Click on the model to start making predictions.

![seeme-ai-your-first-model-cats-dogs](images/seeme-ai-first-model-cats-dogs.png)

Here is what the detail screen looks like:

![SeeMe.ai first model detail screen](images/seeme-ai-model-detail-screen.png)

Next:

- click on `select image`
- find an image you would like to classify
- click on analyze
- Look at `result` and `confidence` to see what the prediction is.

# 3.3 Share your model

Once you have tested your model, it is time to share it with friends.

Go back to the home page, and click the `edit` icon.

![SeeMe.ai edit your model](images/seeme-ai-first-model-cats-dogs-edit.png)

You will go to the model detail screen:

![SeeMe.ai Model detail](images/seeme-ai-model-detail.png)

There you can invite people by entering their email address.

Once invited, they will receive an email to either register (if that email is not yet associated to an account) or to notify of your model being shared with them.

# 3.4 iOS/Android

We are currently also developing the iOS and Android apps. 

The iOS is currently being tested via Apple Testflight. If you would like an invite, you can contact us via [guides@seeme.ai](mailto:guides@seeme.ai).


# 3.5 Other platforms

If there is a particular platform you would live to use to make predictions and the above options don't work for you, we would be happy to hear from you. We offer API and Docker support as well.